In [1]:
import sys, matplotlib, matplotlib.pyplot as plt, pickle, os, numpy as np
sys.path.append("../")
from deap import gp
from deap.tools import Logbook
from typing import Callable, TypeVar, Iterable
from run_gp import RunInfo
from shared_tools.make_datasets import x_train, y_train, x_validation, y_validation, x_test, y_test
from simple_pred.make_pset import pset
from shared_tools.toolbox import create_toolbox
from functools import partial
from dataclasses import dataclass
from main import parser
from deap_tree import Tree, TreeDrawer
from IPython.display import Image

ModuleNotFoundError: No module named 'deap'

In [ ]:
parameters = parser.parse_args([])

datasets = {
    "train": (x_train, y_train),
    "validation": (x_validation, y_validation),
    "test": (x_test, y_test)
}

pset = pset()

toolbox = create_toolbox(datasets, pset, parameters)

In [ ]:
def plot(logbook: Logbook) -> None:
    gen = logbook.select("gen")

    size_avgs = logbook.chapters["size"].select("avg")

    fig, axs = plt.subplots(2, 2)
    combinations = zip(["min", "max", "avg"], ["red", "green", "blue"])

    for ax, (setting, color) in zip(axs.flat, combinations):
        ax.plot(gen, logbook.chapters["fitness"].select(setting),
                color=color, label=f"fitness_{setting}")
        ax.set_title(f"Fitness {setting}")
        ax.set(xlabel='x-label', ylabel='y-label')
    axs[1,1].plot(gen, size_avgs, color="orange")
    axs[1,1].set_title("Average Size")
    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.set(xlabel="generations", ylabel="fitness")
        # ax.label_outer()
    axs[1,1].set(ylabel="size")
    plt.show()


In [ ]:
T = TypeVar('T')
def retrieve_from_files(extractor: Callable[[RunInfo], T], files: Iterable[str],  exclude_zero: bool=True) -> list[T]:
    result = []
    for file_path in files:
        with open(file_path, 'rb') as file:
            try:
                run_info = pickle.load(file)
                if exclude_zero and run_info.parameters.seed == 0:
                    continue
                result.append(extractor(run_info))
            except:
                pass
    return result

def get_files(model: str) -> list[str]:
    return [f"{model}/data/{name}" for name in os.listdir(f"{model}/data")]
    
def plot_across_all_models(
        model: str,
        *extractors: tuple[Callable[[RunInfo], T], str],
        title: str,
        x_axis: str="",
        y_axis: str="",
        exclude_zero: bool=True
    ) -> None:
    for extractor, name in extractors:
        results = np.array(retrieve_from_files(
            extractor,
            (f"{model}/data/{name}" for name in os.listdir(f"{model}/data")),
            exclude_zero=exclude_zero
        ))
        for result in results:
            plt.plot(result, alpha=0.3, label='_nolegend_')
        plt.plot(results.mean(axis=0), linewidth=3,  label=f"{name} average")
    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.legend()
    plt.show()

def box_plot(
        model: str,
        *extractors: tuple[Callable[[RunInfo], T], str],
        title: str,
        x_axis: str="",
        y_axis: str="",
        exclude_zero: bool=True
    ) -> None:
    for extractor, name in extractors:
        results = np.array(retrieve_from_files(
            extractor,
            (f"{model}/data/{name}" for name in os.listdir(f"{model}/data")),
            exclude_zero=exclude_zero
        ))
        plt.boxplot(results)
    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.legend()
    plt.show()

In [ ]:
best_run = min(retrieve_from_files(lambda r: r, get_files('simple_pred')), key=lambda r: r.test_error)
best_individual, wrapper_model = best_run.best_individual, best_run.wrapper_model
print(best_individual)

In [ ]:
plot_across_all_models(
    'simple_pred',
    (lambda i: i.log.select("fit_min"), "fitness"), (lambda i: i.log.select("val_min"), "validation"),
    title=f"Minimum Test and Validation error over every {best_run.model} run",
    x_axis="generations",
    y_axis="OOB error"
)
#plot_across_all_models('simple_pred', lambda i: i.log.select("val_min"), title="minimum validation error simple pred")
box_plot('simple_pred', (lambda i: i.test_error, "test error"), title=f"Test error for all the {best_run.model} models", y_axis="test error")

results = np.array(retrieve_from_files(lambda i: i.test_error, get_files("simple_pred")))
print("Avg test error:", f"{results.mean()} +- {results.std()}")
print("Min test error:", results.min())

In [ ]:
def show(a_v_pairs: list[tuple[float, float]], title: str) -> None:
    aro, val = sum(a for a, _ in a_v_pairs) / len(a_v_pairs), sum(v for _, v in a_v_pairs) / len(a_v_pairs)
    plt.scatter([a for a, _ in a_v_pairs], [v for _, v in a_v_pairs],  s=20, color=(0.1, 0.1, 1, 0.7))
    plt.scatter([aro], [val], s=100, color=(1, 0, 0, 0.7))
    plt.xlabel("arousal")
    plt.ylabel("valence")
    plt.title(title)
    plt.xlim((-1, 1))
    plt.ylim((-1, 1))
    plt.show()


In [ ]:
show(y_validation, title="True distribution of the validation set")
show(y_train, title="True distribution of the train set")
show(y_test, title="True distribution of the test set")

In [ ]:
feature_extractor = toolbox.compile(best_individual)
val_features = np.array([feature_extractor(img) for img in x_validation])
train_features = np.array([feature_extractor(img) for img in x_train])
wrapper_model.fit(train_features, y_train)
val_predictions, train_predictions = wrapper_model.predict(val_features), wrapper_model.predict(train_features)
show(val_predictions, title=f"best {best_run.model} validation predictions")
show(train_predictions, title=f"best {best_run.model} training predictions")

In [ ]:
f = Tree.of(best_individual, pset)
for i, img in enumerate(x_train[:5]):
    TreeDrawer().register_draw_function(lambda t: len(t.value) > 5,  lambda g, t: g.add_node(f"{t.id()}result", label=f"Array with {len(t.value)} elements", shape="plaintext")).save_graph(f"models/best_model{i}.png", f, img)
    display(Image(f"models/best_model{i}.png"))